# Downloading dataset

In [0]:
! wget https://archive.org/download/mnist/mnist.zip

# Extracting Dataset

In [0]:
! unzip mnist.zip -d mnist/

# Loading dataset in memory

In [0]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical

In [0]:
df_train = pd.read_csv("mnist/train.csv")
X_train = df_train.iloc[:,1:].values
Y_train = df_train.iloc[:,0].values
Y_train = to_categorical(Y_train, num_classes=10) # one hot encoding of labels

# Installing boto3 package too upload files to AWS S3 or Digital Ocean Spaces

In [0]:
! pip install boto3
import boto3

# Creating model

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [0]:
def create_model():
  model = Sequential()
  model.add(Dense(30,input_dim=784,activation='sigmoid'))
  model.add(Dense(50,activation='sigmoid'))
  model.add(Dense(50,activation='sigmoid'))
  model.add(Dense(10,activation='softmax'))

  model.compile(loss="categorical_crossentropy", optimizer='adam',metrics=['accuracy'])
  model.summary()
  return model

# Training model

#### Make sure to update value of parameters used below

In [0]:
model = create_model()
! mkdir models
no_of_epochs = 10
session = boto3.Session(aws_access_key_id="Put AWS Key ID here " , aws_secret_access_key = " Put secret key here" )
s3 = session.resource('s3',endpoint_url="you can configure this endpoint for using digital ocean spaces, remove this option if using s3 ",region_name=" set region name of bucket")
bucket = s3.Bucket("Bucket Name")

for i in range(no_of_epochs):
  history = model.fit(X_train,Y_train,epochs=1)
  file_name = "models/" + "epoch-" + str(i) + "-loss-" + str(history.history["loss"][0]) + ".h5"
  # saving model
  model.save(file_name)
  
  # compressing files
  ! tar -cvf models.tar.gz models
  
  # uploading compressed models.tar.gz to s3 bucket and making it public
  with open('models.tar.gz','rb') as data:
    bucket.upload_fileobj(data, 'models.tar.gz')
  object_acl = s3.ObjectAcl('Bucket name','models.tar.gz')
  response = object_acl.put(ACL='public-read') # to make the file public so that you can download it using wget commands whenever you train next time
  
! ls models/

In [0]:
model.evaluate(X_train,Y_train)